In [ ]:
%pip install langchain
%pip install sentence-transformers


In [ ]:
%pip install -U langchain-community

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
  model_name="intfloat/multilingual-e5-large"
)

In [ ]:
%pip install nltk

In [ ]:
%pip install bertopic
%pip install sentence-transformers umap-learn hdbscan

In [ ]:
import pandas as pd
from bertopic import BERTopic
import random
#from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import torch
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("Sentiment/Datensammlung.csv")

C:\Users\denni\AppData\Local\Temp\ipykernel_5632\4132829028.py:1: DtypeWarning:

Columns (1,29) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#removing posts without text
df = df[df['body'].notna()]

In [3]:
stop = set(stopwords.words("german"))

Function to remove stopwords from the posts

In [6]:
def remove_stopwords(text):
    # Tokenisiere den Text
    tokens = word_tokenize(text, language='german')
    # Entferne Satzzeichen
    tokens = [word for word in tokens if word.isalpha()]
    # Entferne die Stoppwörter
    filtered_tokens = [word for word in tokens if word.lower() not in stop]
    # Gib den gefilterten Text als String zurück
    return ' '.join(filtered_tokens)

In [ ]:
stop = stopwords.words("german")
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop)

In [7]:
df['body_without_stopwords'] = df['body'].apply(remove_stopwords)

In [ ]:
df.to_csv("Datensammlung_ohne_Stopwörter.csv", encoding="utf-8")

In [ ]:
df = df[df['body_without_stopwords'].notna()]

In [ ]:
df.to_csv("Datensammlung_ohne_Stopwörter2.csv", encoding="utf-8")

Here beginns the actual training of the model

In [ ]:
import pandas as pd
from bertopic import BERTopic
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import torch
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv("Datensammlung_ohne_Stopwörter2.csv", encoding="utf-8")
#this csv is a version of the dataframe were after removing the stopwords from the

C:\Users\denni\AppData\Local\Temp\ipykernel_624\3007582433.py:1: DtypeWarning:

Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
#adding documents to list
texts = df["body_without_stopwords"].tolist()

In [6]:
stop = stopwords.words("german")

In [6]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop)

Creating prior embeddings to save time and ressources for the actual training

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = embedding_model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/20439 [00:00<?, ?it/s]

In [ ]:
with open('embeddings.npy', 'wb') as f:
    np.save(f, embeddings)

In [6]:
embeddings = np.load('embeddings.npy')

In [9]:
from sentence_transformers import SentenceTransformer

Training the model

In [10]:
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
topic_model = BERTopic(language="multilingual", calculate_probabilities=False, verbose=True, vectorizer_model=vectorizer_model, embedding_model=embedding_model).fit(texts, embeddings=embeddings)

2025-04-11 19:09:36,909 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-11 19:19:13,856 - BERTopic - Dimensionality - Completed ✓
2025-04-11 19:19:13,874 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-11 19:21:17,989 - BERTopic - Cluster - Completed ✓
2025-04-11 19:21:18,136 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-11 19:23:38,136 - BERTopic - Representation - Completed ✓


Through every step the model is saved as a backup

In [11]:
topic_model.save("my_bertopic_model_vorerst")

2025-04-11 19:27:16,807 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [9]:
topic_model = BERTopic.load("my_bertopic_model_vorerst")

In [ ]:
topics, probs = topic_model.fit_transform(texts)

In [9]:
topic_model.reduce_topics(texts, nr_topics= 150)

2025-04-11 20:14:23,082 - BERTopic - Topic reduction - Reducing number of topics
2025-04-11 20:14:23,818 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-11 20:16:35,114 - BERTopic - Representation - Completed ✓
2025-04-11 20:16:35,568 - BERTopic - Topic reduction - Reduced number of topics from 3387 to 150


In [10]:
topic_model.save("my_bertopic_model_mini")

2025-04-11 20:16:58,370 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [4]:
topic_model = BERTopic.load("my_bertopic_model_mini")

After fitting the models, the outlier are now reduced

In [ ]:
topics, probs = topic_model.fit_transform(texts)
new_topics = topic_model.reduce_outliers(texts, topics)

In [ ]:
topic_model.save("my_bertopic_model_outlier_reduced")

2025-04-12 21:33:08,917 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
new_topics, _ = topic_model.transform(texts)

In [ ]:
unique_topics = sorted(set(new_topics))

The new mapping of the topics is added

In [8]:
topic_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_topics)}

In [ ]:
df["topic"] = [topic_mapping[t] for t in new_topics]


In [11]:
df.to_csv("Datensammlung_plus_Topic.csv", encoding="utf-8")

Model is finished and the dataframe saved

Here the informations to the model are saved

In [12]:
info = topic_model.get_topic_info()

In [16]:
info.to_csv("Model_Info_in_df.csv")